In [1]:
# Import dependencies
import pandas as pd

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Read the CSV and Perform Basic Data Cleaning

In [2]:
# Read in data
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
print(df["koi_disposition"].unique())

['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']


# Select features (columns)

In [4]:
# Set X equal to the entire data set, except for the first column
X = df.iloc[:,1:]

# Set y equal to the first column
y = df.iloc[:,0]

In [5]:
# Search for top 10 features according to feature importances
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X,y)
model.feature_importances_

array([0.11651221, 0.15975025, 0.13329999, 0.0534179 , 0.01663236,
       0.01663748, 0.01664759, 0.01303336, 0.01210654, 0.02539675,
       0.01096942, 0.00920767, 0.01126036, 0.01832687, 0.02592463,
       0.03295711, 0.02331014, 0.01441164, 0.02035426, 0.01771513,
       0.0157412 , 0.01260404, 0.01513189, 0.00917879, 0.00824968,
       0.00837358, 0.02459698, 0.00673734, 0.00898861, 0.0283035 ,
       0.01921845, 0.01034994, 0.01117716, 0.01476235, 0.00906196,
       0.00901745, 0.00912011, 0.00980477, 0.01018609, 0.01152446])

In [6]:
# Store the top 10 features as a series, using the column headers as the index
feat_imp = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
feat_imp

koi_fpflag_ss        0.159750
koi_fpflag_co        0.133300
koi_fpflag_nt        0.116512
koi_fpflag_ec        0.053418
koi_duration_err2    0.032957
koi_steff_err1       0.028303
koi_duration_err1    0.025925
koi_time0bk_err2     0.025397
koi_model_snr        0.024597
koi_depth            0.023310
dtype: float64

In [7]:
# Set features based on feature importances
X = df[feat_imp.index]

# Use koi_disposition for y values
y = df['koi_disposition']

# Create a Train Test Split

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler

In [9]:
from sklearn.preprocessing import MinMaxScaler

# Create scaler object
X_scaler = MinMaxScaler().fit(X_train)

# Scale training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model
Using Random Forest


In [10]:
from sklearn.ensemble import RandomForestClassifier

# Initialize model (random forest)
rf = RandomForestClassifier(n_estimators=300)

# Train the model
rf = rf.fit(X_train_scaled, y_train)

In [11]:
# Score the model
print(f"Training Data Score: {rf.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8718535469107551


# Predict

In [12]:
predictions = rf.predict(X_test_scaled)
df_pred = pd.DataFrame({"Actual":y_test, "Predicted":predictions}) 
df_pred.head()

,Actual,Predicted
196,CONFIRMED,CONFIRMED
2658,CONFIRMED,CONFIRMED
6787,FALSE POSITIVE,FALSE POSITIVE
36,CONFIRMED,CONFIRMED
5062,FALSE POSITIVE,FALSE POSITIVE


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [13]:
from sklearn.model_selection import GridSearchCV

# Create the GridSearchCV model
param_grid = {'n_estimators':[200, 400, 600, 800, 1000],
             'max_depth':[5, 8, 15, 25, 30],
             'min_samples_split':[2, 5, 10],
             'min_samples_leaf':[1, 2, 4]}

grid = GridSearchCV(rf, param_grid, verbose=3)

In [14]:
# Train the model with GridSearch
best_model = grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 225 candidates, totalling 675 fits
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200, score=0.890, total=   1.4s
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


[CV]  max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200, score=0.877, total=   1.4s
[CV] max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200 


...

[CV]  max_depth=25, min_samples_leaf=4, min_samples_split=2, n_estimators=200, score=0.891, total=   0.9s
[CV] max_depth=25, min_samples_leaf=4, min_samples_split=2, n_estimators=400 
[CV]  max_depth=25, min_samples_leaf=4, min_samples_split=2, n_estimators=400, score=0.893, total=   1.9s
[CV] max_depth=25, min_samples_leaf=4, min_samples_split=2, n_estimators=400 
[CV]  max_depth=25, min_samples_leaf=4, min_samples_split=2, n_estimators=400, score=0.887, total=   1.9s
[CV] max_depth=25, min_samples_leaf=4, min_samples_split=2, n_estimators=400 
[CV]  max_depth=25, min_samples_leaf=4, min_samples_split=2, n_estimators=400, score=0.890, total=   1.9s
[CV] max_depth=25, min_samples_leaf=4, min_samples_split=2, n_estimators=600 
[CV]  max_depth=25, min_samples_leaf=4, min_samples_split=2, n_estimators=600, score=0.894, total=   2.8s
[CV] max_depth=25, min_samples_leaf=4, min_samples_split=2, n_estimators=600 
[CV]  max_depth=25, min_samples_leaf=4, min_samples_split=2, n_estimators=600, s

[CV]  max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=200, score=0.890, total=   1.1s
[CV] max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400 
[CV]  max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400, score=0.891, total=   2.1s
[CV] max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400 
[CV]  max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400, score=0.883, total=   2.0s
[CV] max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400 
[CV]  max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=400, score=0.887, total=   2.1s
[CV] max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=600 
[CV]  max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=600, score=0.888, total=   3.1s
[CV] max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=600 
[CV]  max_depth=30, min_samples_leaf=1, min_samples_split=2, n_estimators=600, s

[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=200, score=0.890, total=   1.0s
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=400 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=400, score=0.890, total=   2.0s
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=400 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=400, score=0.883, total=   2.0s
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=400 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=400, score=0.888, total=   2.0s
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=600 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=600, score=0.892, total=   2.9s
[CV] max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=600 
[CV]  max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=600, s

[CV]  max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=200, score=0.887, total=   1.0s
[CV] max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=400 
[CV]  max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=400, score=0.894, total=   1.9s
[CV] max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=400 
[CV]  max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=400, score=0.883, total=   1.9s
[CV] max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=400 
[CV]  max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=400, score=0.888, total=   1.9s
[CV] max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=600 
[CV]  max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=600, score=0.897, total=   2.8s
[CV] max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=600 
[CV]  max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=600, s

[Parallel(n_jobs=1)]: Done 675 out of 675 | elapsed: 35.1min finished


In [15]:
print(best_model.best_params_)
print(best_model.best_score_)

{'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 400}
0.8916650772458516


In [16]:
# Assign best parameters to variables
best_depth = best_model.best_params_['max_depth']
best_leaf = best_model.best_params_['min_samples_leaf']
best_split = best_model.best_params_['min_samples_split']
best_estimators = best_model.best_params_['n_estimators']

In [17]:
# Make predictions
grid_predictions = best_model.predict(X_test_scaled)
df_grid = pd.DataFrame({"Actual":y_test, "Predicted":grid_predictions})
df_grid.head()

,Actual,Predicted
196,CONFIRMED,CONFIRMED
2658,CONFIRMED,CONFIRMED
6787,FALSE POSITIVE,FALSE POSITIVE
36,CONFIRMED,CONFIRMED
5062,FALSE POSITIVE,FALSE POSITIVE


# Predict With GridSearch Parameters


In [18]:
# Initialize model with new parameters
rf = RandomForestClassifier(n_estimators=best_estimators, 
                            max_depth=best_depth,
                            min_samples_leaf=best_leaf,
                            min_samples_split=best_split)

# Train the model
rf.fit(X_train_scaled, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=15, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [19]:
# Make predictions on test data
df2 = pd.DataFrame({'Actual':y_test, 'Predicted':rf.predict(X_test_scaled)})
df2.head()

,Actual,Predicted
196,CONFIRMED,CONFIRMED
2658,CONFIRMED,CONFIRMED
6787,FALSE POSITIVE,FALSE POSITIVE
36,CONFIRMED,CONFIRMED
5062,FALSE POSITIVE,FALSE POSITIVE


In [21]:
print(rf.score(X_test_scaled, y_test))

0.8764302059496567
